### Pretendo cruzar los datos de las poblaciones donde se realizan los festivales de música con los datos demográficos/ geográficos (nº de habitantes, província costera, comunidad autónoma) de las localidades donde se celebra el evento para tratar de extraer conclusiones al respecto.

In [1]:
import pandas as pd
import re
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

Importo dos csv (df_festivales) y (df_fest_enlaces). En cada uno de los csv tengo información de los festivales, la cual se complementa. Ambos archivos los he extraido a través deL scrapping pero no he recopilado toda la información junta y en un sólo fichero, por lo que uniré la información de ambos ficheros para tener el dataframe con la información completa.

In [2]:
df_fest = pd.read_csv(r"..\data\datos_demograficos\df_festivales.csv", encoding= "utf16", index_col= "Unnamed: 0")
df_fest_enlaces = pd.read_csv(r"..\data\datos_demograficos\df_festivales_enlaces_gratuito.csv", encoding= "utf16", index_col= "Unnamed: 0")

In [3]:
df_fest.head(3)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos
0,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190.0,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']"
1,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570.0,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']"
2,Locos por la Música 2023 (Fuengirola),Marenostrum Fuengirola,2023-07-29,2023-07-29,1,29640.0,36.524777,-4.628699,Málaga,Fuengirola,"['Melón Diesel', 'No Me Pises Que Llevo Chancl...","['rock', 'pop']"


In [5]:
df_fest_enlaces.head(3)

,enlace_foto,cartel_peq,gratuito,nombre
0,https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,No Future Fest 2023 (Vol.5)
1,https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...
2,https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,Locos por la Música 2023 (Fuengirola)


Uno los dataframes para tener toda la información junta

In [6]:
df_fest = pd.merge(df_fest, df_fest_enlaces, how= "left" ,left_on= "festival", right_on= "nombre")
df_fest = df_fest.rename(columns={0: 'artistas'})
df_fest = df_fest.drop(columns= "nombre")
df_fest

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito
0,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190.0,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
1,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570.0,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
2,Locos por la Música 2023 (Fuengirola),Marenostrum Fuengirola,2023-07-29,2023-07-29,1,29640.0,36.524777,-4.628699,Málaga,Fuengirola,"['Melón Diesel', 'No Me Pises Que Llevo Chancl...","['rock', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
3,A Summer Story 2023,Ciudad del Rock,2023-06-23,2023-06-24,2,28500.0,40.259650,-3.405805,Madrid,Arganda del Rey,"['Armin van Buuren', 'Don Diablo', 'DJ Nano', ...","['electronic', 'techno', 'house', 'dance', 'EDM']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
4,Love To Rock 2023 (Valencia),La Marina de Valencia,2023-10-13,2023-10-14,2,NaN,39.461771,-0.319321,Valencia,Valencia,"['Quique González', 'Cariño', 'Joe Crepúsculo'...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,Valladolid Blues 2023,Teatro Carrión,2023-02-10,2023-03-10,29,47001.0,41.649996,-4.728177,Valladolid,Valladolid,"['Martín Burguez Trío', 'Mingo Balaguer', ""Tra...",['blues'],NaN,NaN,NaN
410,Carrion Rock Festival 2023,Sala Infinity,2023-02-11,2023-02-11,1,34004.0,42.006608,-4.519082,Palencia,Palencia,"['Guadaña', 'Ciclón', 'Invadeath']","['rock', 'metal', 'heavy metal']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
411,Wáchina Wáchina Fest 2023,16 Toneladas,2023-02-18,2023-02-18,1,46009.0,39.481255,-0.387633,Valencia,Valencia,"['The Mourning After', 'The Mocks', 'MOOON', '...","['punk rock', 'rock n roll', 'garage rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
412,Black Sound Fest 2023,Teatro Imperial,2023-02-04,2023-02-25,22,6400.0,38.953452,-5.860906,Badajoz,Don Benito,"['Vargas Blues Band', 'Gonzalo Portugal', 'The...","['rock', 'blues']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False


Veo que existen nulos que tendré que corregir. En lo que más interés tengo es en completar los registros que no tienen código pstal. Lo necesitaré para cruzarlo con los datos de habitantes de cada localidad. 

In [7]:
df_fest.isnull().sum().sort_values(ascending=False)

cod_postal      51
enlace_foto     41
cartel_peq      41
gratuito        41
provincia        7
localidad        3
lugar            1
festival         0
fecha_inicio     0
fecha_fin        0
duracion         0
latitud          0
longitud         0
artistas         0
estilos          0
dtype: int64

Defino un par de funciones que me ayude a modificar los valores erróneos que encuentre en el dataframe

In [8]:
def change_values_columns_2(dicc, df, column1, column2):
    '''
    Función que corrige valores erróneos ya sea por errores ortográficos errores a la hora de almacenar los datos de la web etc...

    input: recibe un diccionario (dict) con los valores a cambiar y los correctos un datafreme (df) sobre el que iterar y modificar los valores
    y las columnas (str) sobre la que se quiere realizar el cambio.

    output: retorna el dataframe modificado aunque las modificaciones ya se hayan realizado
    '''  
    for name_col1, name_col2 in dicc.items():
            for index, row in df.iterrows():
                if row[column1]== name_col1:
                    df.at[index, column2] = name_col2

In [9]:
def change_values_column1(dicc, df, column):
    '''
    Función que corrige valores erróneos ya sea por errores ortográficos errores a la hora de almacenar los datos de la web etc...

    input: recibe un diccionario (dict) con los valores a cambiar y los correctos un datafreme (df) sobre el que iterar y modificar los valores
    y la columna (str) sobre la que se quiere realizar el cambio.

    output: retorna el dataframe modificado aunque las modificaciones ya se hayan realizado
    '''  
    for name_bad, name_good in dicc.items():
            for index, row in df.iterrows():
                if row[column] == name_bad:
                    df.at[index, column] = name_good

En la columna de "provincia" tengo 7 nulos

In [10]:
len(df_fest[df_fest["provincia"].isnull().sort_values(ascending=False)])

7

In [11]:
df_fest[df_fest["provincia"].isnull().sort_values(ascending=False)]

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito
22,OFFSónar 2023,Poble Espanyol,2023-06-15,2023-06-18,4,8038.0,41.368544,2.148258,NaN,Barcelona,sin artistas,"['electronic', 'dj', 'techno', 'house', 'elect...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
57,Iberojoven 2023,Recinto IberoJoven,2023-04-01,2023-04-01,1,23460.0,37.917058,-3.119613,NaN,Peal de Becerro,"['Fangoria', 'Álvaro de Luna', 'Nancys Rubias'...","['rock', 'pop', 'dj', 'fusion']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
66,Womad Cáceres 2023,Plaza de San Jorge,2023-05-11,2023-05-14,4,NaN,39.476179,-6.370760,NaN,Cáceres,sin artistas,"['rock', 'indie pop', 'folk', 'fusion', 'latin']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,True
99,Concert Music Festival 2023,Concert Music Festival,2023-07-01,2023-08-31,62,11139.0,36.394005,-6.207314,NaN,Chiclana de la Frontera (poblado de Sancti Petri),"['Alejandro Sanz', 'David Bisbal', 'Bizarrap',...","['rock', 'pop', 'folk', 'fusion', 'flamenco']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
167,Microsonidos 2023,Varias localizaciones,2023-02-03,2023-04-07,64,NaN,37.992050,-1.125357,NaN,Murcia,"['Rufus T. Firefly', 'Biznaga', 'Las Wonder', ...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
180,En Orbita 2023,Pendiente confirmación,2023-05-05,2023-05-06,2,NaN,0.000000,0.000000,NaN,NaN,sin artistas,"['rock', 'indie', 'pop', 'indie pop', 'indie r...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
340,Surf O Rama Festival 2023,Sala Repvblicca,2023-06-02,2023-06-03,2,46920.0,39.472830,-0.427006,NaN,Valencia,"['Wau y los Arrrghs!!!', 'Los Tiki Phantoms', ...","['rock', 'rock n roll', 'rockabilly', 'surf', ...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False


Con la ayuda de la función asigno los valores correctos

In [12]:
dict_provincia= {
                "OFFSónar 2023": "Barcelona",
                "Iberojoven 2023": "Jaén",
                "Womad Cáceres 2023": "Cáceres",
                "Concert Music Festival 2023": "Barcelona",
                "Microsonidos 2023": "Cádiz",
                "En Orbita 2023": "Murcia",
                "Surf O Rama Festival 2023": "Valencia",
                "Microsonidos 2023": "Murcia"
                }
change_values_columns_2(dict_provincia, df_fest, "festival", "provincia")

En la columna de "localidad" tengo 3 nulos

In [13]:
len(df_fest[df_fest["localidad"].isnull().sort_values(ascending=False)])

3

In [14]:
df_fest[df_fest["localidad"].isnull().sort_values(ascending=False)]

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito
180,En Orbita 2023,Pendiente confirmación,2023-05-05,2023-05-06,2,NaN,0.000000,0.000000,Murcia,NaN,sin artistas,"['rock', 'indie', 'pop', 'indie pop', 'indie r...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
232,Prestoso Fest 2023,Las Bazaniellas,2023-08-03,2023-08-05,3,33811.0,43.157850,-6.540406,Asturias,NaN,"['Coach Party', 'Rodrigo Cuevas', '_juno', 'Ai...","['indie', 'indie pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
286,Reggaeton Beach Festival (Tenerife) 2023,La Laguna,2023-07-01,2023-07-02,2,NaN,28.487002,-16.321406,Santa Cruz de Tenerife,NaN,['Manuel Turizo'],"['urban', 'latin', 'reggaeton']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False


Corrijo estos registros y aprovecho a corregir otros errores en el nombre de la localidad (localidades escritas de forma diferentes "tema acentos, diferentes dialectos")

In [15]:
dict_localidad= {
                "En Orbita 2023": "Armilla",
                "Prestoso Fest 2023": "Cangas del Narcea",
                "Reggaeton Beach Festival (Tenerife) 2023": "San Cristóbal de la Laguna",
                "Isla Bonita Love Festival 2023": "Tazacorte",
                "Unirock 2023": "Navia",
                "Montgorock (MRK Xàbia Festival) 2023": "Jávea",
                "Sansan Festival 2023": "Benicássim",
                "FIB 2023": "Benicássim",
                "Full Metal Holiday 2023 (Destination Mallorca)": "Mallorca",
                "Sierra Nevada por Todo lo Alto 2023": "Monachil",
                "Heavy Jaia 2023": "San Sebastián",
                "Concert Music Festival 2023": "Chiclana de la Frontera",
                "Mad Cool Festival 2023": "Madrid",
                "Rototom Sunsplash 2023": "Benicássim"
                }
change_values_columns_2(dict_localidad, df_fest, "festival", "localidad")

En la columna de "cod_postal" tengo 51 nulos

In [16]:
len(df_fest[df_fest["cod_postal"].isnull().sort_values(ascending=False)])

51

Muestro los 5 primeros de los 51 registros que no tienen código postal

In [17]:
df_fest[df_fest["cod_postal"].isnull().sort_values(ascending=False)].head(5)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito
4,Love To Rock 2023 (Valencia),La Marina de Valencia,2023-10-13,2023-10-14,2,NaN,39.461771,-0.319321,Valencia,Valencia,"['Quique González', 'Cariño', 'Joe Crepúsculo'...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
8,NEU! Festival 2022/2023,Varias localizaciones,2022-11-26,2023-01-28,64,NaN,41.978667,2.821228,Girona,Girona,"['Carolina Durante', 'Mishima', 'Chaqueta de C...","['indie', 'pop', 'folk', 'singer-songwriter']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
10,Festival Millenni 2023,Varias localizaciones,2023-01-21,2023-12-30,344,NaN,41.387579,2.175304,Barcelona,Barcelona,"['Fito y Fitipaldis', 'Hombres G', 'Amaia', 'C...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
33,FIB 2023,Benicàssim,2023-07-13,2023-07-16,4,NaN,40.048068,0.047872,Castellón,Benicássim,"['Franz Ferdinand', 'Bastille', 'The Offspring...","['rock', 'indie', 'pop', 'electronic', 'altern...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
35,Monegros Desert Festival 2023,Desierto de los Monegros,2023-07-29,2023-07-29,1,NaN,41.524747,0.125433,Huesca,Fraga,"['Fernandocosta', 'Iseo & Dodosound', 'Oscar M...","['electronic', 'dj', 'hip-hop', 'techno', 'hou...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False


Corrijo algunos registros de estos 51

In [18]:
dict_cod_postal= {
                "En Orbita 2023": "18100", #Feria de Muestras de Armilla, Granada
                "Prestoso Fest 2023": "33800",
                "Reggaeton Beach Festival (Tenerife) 2023": "38291"
                }
change_values_columns_2(dict_cod_postal, df_fest, "festival", "cod_postal")

Corrijo latitud y longitud de otro registro erróneo

In [21]:
dict_cod_latitud= {
                "En Orbita 2023": -3.6188300,
                }

dict_cod_longitud= {
                "En Orbita 2023":  37.1509700,
                }

change_values_columns_2(dict_cod_latitud, df_fest, "festival", "latitud")
change_values_columns_2(dict_cod_longitud, df_fest, "festival", "longitud")

Registro sin la columna "lugar" completa

In [22]:
df_fest[df_fest["lugar"].isnull().sort_values(ascending=False)]

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito
29,Holika Festival 2023,NaN,2023-06-29,2023-07-01,3,26500.0,42.297183,-1.95797,La Rioja,Calahorra,"['Eladio Carrión', 'Justin Quiles', 'Mora', 'C...","['dj', 'hip-hop', 'techno', 'house', 'urban']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False


In [23]:
dict_lugar= {
                "En Orbita 2023": "Feria de Muestras de Armilla",
                "Holika Festival 2023": "Complejo Deportivo Municipal La Planilla"
                }

change_values_columns_2(dict_lugar, df_fest, "festival", "lugar")

Siguen habiendo 49 registros sin código postal

In [24]:
len(list(df_fest[df_fest["cod_postal"].isnull()].loc[:,["festival", "localidad","provincia", "lugar"]].values))

49

In [25]:
cod_postal_nulo = df_fest[df_fest["cod_postal"].isnull()]
cod_postal_nulo.head(5)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito
4,Love To Rock 2023 (Valencia),La Marina de Valencia,2023-10-13,2023-10-14,2,NaN,39.461771,-0.319321,Valencia,Valencia,"['Quique González', 'Cariño', 'Joe Crepúsculo'...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
8,NEU! Festival 2022/2023,Varias localizaciones,2022-11-26,2023-01-28,64,NaN,41.978667,2.821228,Girona,Girona,"['Carolina Durante', 'Mishima', 'Chaqueta de C...","['indie', 'pop', 'folk', 'singer-songwriter']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
10,Festival Millenni 2023,Varias localizaciones,2023-01-21,2023-12-30,344,NaN,41.387579,2.175304,Barcelona,Barcelona,"['Fito y Fitipaldis', 'Hombres G', 'Amaia', 'C...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
33,FIB 2023,Benicàssim,2023-07-13,2023-07-16,4,NaN,40.048068,0.047872,Castellón,Benicássim,"['Franz Ferdinand', 'Bastille', 'The Offspring...","['rock', 'indie', 'pop', 'electronic', 'altern...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
35,Monegros Desert Festival 2023,Desierto de los Monegros,2023-07-29,2023-07-29,1,NaN,41.524747,0.125433,Huesca,Fraga,"['Fernandocosta', 'Iseo & Dodosound', 'Oscar M...","['electronic', 'dj', 'hip-hop', 'techno', 'hou...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False


Veo que ya he reducido los nulos en provincia, localidad y lugar

In [26]:
df_fest.isnull().sum().sort_values(ascending=False)

cod_postal      49
enlace_foto     41
cartel_peq      41
gratuito        41
festival         0
lugar            0
fecha_inicio     0
fecha_fin        0
duracion         0
latitud          0
longitud         0
provincia        0
localidad        0
artistas         0
estilos          0
dtype: int64

Saco un listado de la localidades sin código postal

In [27]:
list(cod_postal_nulo["localidad"])

['Valencia',
 'Girona',
 'Barcelona',
 'Benicássim',
 'Fraga',
 'Palma',
 'Barcelona',
 'Madrid',
 'Cáceres',
 'Fuengirola',
 'Alboloduy',
 'Oropesa del Mar',
 'Barcelona',
 'Canet de Mar',
 'Tazacorte',
 'Torres',
 'El Vendrell',
 'Grimaldo',
 'Vilagarcía de Arousa',
 'Los Silos',
 'Madrid',
 'Esquivias',
 'Benicássim',
 'Murcia',
 'Gijón',
 'Jávea',
 'Valladolid',
 'Balboa',
 'Arnedo',
 'Vilagarcía de Arousa',
 'Redondela',
 'Ayerbe',
 'Valencia de Don Juan',
 'Burela',
 'Navalmoral de la Mata',
 'Valencia',
 'Sant Adrià de Besòs',
 'Puertollano',
 'Guareña',
 'Torre de Don Miguel',
 'Burgos',
 'Astudillo',
 'Venta de Baños',
 'Intriago',
 'Burgos',
 'Pinofranqueado',
 'Navia',
 'Sopela',
 'Pola de Laviana']

Corrijo los códigos postales de dichas localidades

In [28]:
dict_cod_postal_localidad= {
                            "Pobla de Massaluca, La": "26120",
                            "Fariza": "43783",
                            "Albelda de Iregua": "49213",
                            "Benicássim": "12560",
                            "Valencia": "46001",
                            "Girona": "17001",
                            "Barcelona": "08001",
                            "Palma": "07001",
                            "Madrid": "28001",
                            "Cáceres": "10001",
                            "Fuengirola": "29640",
                            "Alboloduy": "04531",
                            "Oropesa del Mar": "12594",
                            "Canet de Mar": "08360",
                            "Tazacorte": "38770",
                            "El Vendrell": "43700",
                            "Grimaldo": "36600",
                            "Fraga": "22520",
                            "Vilagarcía de Arousa": "36600",
                            "Los Silos": "38435",
                            "Esquivias": "45221",
                            "Murcia": "30001",
                            "Gijón": "33201",
                            "Jávea": "03730",
                            "Valladolid": "47001",
                            "Balboa": "24525",
                            "Arnedo": "26580",
                            "Redondela": "36800",
                            "Ayerbe": "22800",
                            "Valencia de Don Juan": "24200",
                            "Burela": "27880",
                            "Navalmoral de la Mata": "10300",
                            "Sant Adrià de Besòs": "08020",
                            "Puertollano": "13500",
                            "Guareña": "06470",
                            "Torre de Don Miguel": "10864",
                            "Burgos": "09001",
                            "Astudillo": "34450",
                            "Venta de Baños": "34200",
                            "Intriago": "33556",
                            "Pinofranqueado": "10630",
                            "Navia": "33710",
                            "Sopela": "48600",
                            "Pola de Laviana": "33980",
                            "Torres": "23540"
                            }
change_values_columns_2(dict_cod_postal_localidad, df_fest, "localidad", "cod_postal")

Compruebo que ya no tengo nulos en la columna cod_postal. Las columnas enlace_foto, cartel_peq y gratuito siguen teniendo nulos pero no es importante para proseguir con el análisis

In [29]:
df_fest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 0 to 413
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   festival      414 non-null    object 
 1   lugar         414 non-null    object 
 2   fecha_inicio  414 non-null    object 
 3   fecha_fin     414 non-null    object 
 4   duracion      414 non-null    int64  
 5   cod_postal    414 non-null    object 
 6   latitud       414 non-null    float64
 7   longitud      414 non-null    float64
 8   provincia     414 non-null    object 
 9   localidad     414 non-null    object 
 10  artistas      414 non-null    object 
 11  estilos       414 non-null    object 
 12  enlace_foto   373 non-null    object 
 13  cartel_peq    373 non-null    object 
 14  gratuito      373 non-null    object 
dtypes: float64(2), int64(1), object(12)
memory usage: 67.9+ KB


Importo csv con todas las localidades de España donde tienen el código postal y código de municipio (datos que necesitaré para cruzarlo con dsatos de número de habitantes de localidades de toda España que importaré posteriormente)

Fuente: https://www.ine.es/ss/Satellite?L=es_ES&c=Page&cid=1259952026632&p=1259952026632&pagename=ProductosYServicios%2FPYSLayout. Callejero del Censo electoral (INE)

In [31]:
cod_postales = pd.read_csv(r"..\data\datos_demograficos\codigos_postales.csv")
cod_postales

,codigo_postal,municipio_id,municipio_nombre,codigo_unidad_poblacional,entidad_singular_nombre,nucleo_nombre
0,43,43110,"Pobla de Massaluca, La",1799,Pobla de Massaluca (La),*Diseminado*
1,85,49064,Fariza,5699,Palazuelo de Sayago,*Diseminado*
2,633,26005,Albelda de Iregua,1799,Albelda de Iregua,*Diseminado*
3,1001,1059,Vitoria-Gasteiz,63501,Vitoria-Gasteiz,Vitoria-Gasteiz
4,1002,1059,Vitoria-Gasteiz,63501,Vitoria-Gasteiz,Vitoria-Gasteiz
...,...,...,...,...,...,...
77615,52002,52001,Melilla,3801,Melilla,Melilla
77616,52003,52001,Melilla,3801,Melilla,Melilla
77617,52004,52001,Melilla,3801,Melilla,Melilla
77618,52005,52001,Melilla,3801,Melilla,Melilla


Cambio el tipo de dato de algunas de las columnas a string para poder modificar código postal y código municipio ya que se ha prescindido de los ceros del inicio de dichos códigos

In [32]:
cod_postales["codigo_postal"] = cod_postales["codigo_postal"].astype(str)
cod_postales["municipio_id"] = cod_postales["municipio_id"].astype(str)

for index, row in cod_postales.iterrows():
    if len(row["codigo_postal"]) == 4:
        cod_postales.at[index, "codigo_postal"] = "0" + row["codigo_postal"]

for index, row in cod_postales.iterrows():
    if len(row["municipio_id"]) == 4:
        cod_postales.at[index, "municipio_id"] = "0" + row["municipio_id"]

Corrijo algunos registros que están erróneos

In [33]:
dict_cod_postal_localidad= {
                "Pobla de Massaluca, La": "26120",
                "Fariza": "43783",
                "Albelda de Iregua": "49213"
                }
change_values_columns_2(dict_cod_postal_localidad, cod_postales, "municipio_nombre", "codigo_postal")

In [86]:
cod_postales

,codigo_postal,municipio_id,municipio_nombre,codigo_unidad_poblacional,entidad_singular_nombre,nucleo_nombre
0,26120,43110,"Pobla de Massaluca, La",1799,Pobla de Massaluca (La),*Diseminado*
1,43783,49064,Fariza,5699,Palazuelo de Sayago,*Diseminado*
2,49213,26005,Albelda de Iregua,1799,Albelda de Iregua,*Diseminado*
3,01001,01059,Vitoria-Gasteiz,63501,Vitoria-Gasteiz,Vitoria-Gasteiz
4,01002,01059,Vitoria-Gasteiz,63501,Vitoria-Gasteiz,Vitoria-Gasteiz
...,...,...,...,...,...,...
77615,52002,52001,Melilla,3801,Melilla,Melilla
77616,52003,52001,Melilla,3801,Melilla,Melilla
77617,52004,52001,Melilla,3801,Melilla,Melilla
77618,52005,52001,Melilla,3801,Melilla,Melilla


Importo csv con la información de el nº de habitantes de cada localidad de España.

Fuente: https://www.ine.es/dynt3/inebase/es/index.htm?padre=525

In [34]:
poblacion_municipios = pd.read_excel(r"..\data\datos_demograficos\censo_localidades.xlsx")
poblacion_municipios

,CPRO,PROVINCIA,CMUN,NOMBRE,POB22,HOMBRES,MUJERES
0,2,Albacete,1,Abengibre,739,363,376
1,2,Albacete,2,Alatoz,496,276,220
2,2,Albacete,3,Albacete,172357,84230,88127
3,2,Albacete,4,Albatana,670,342,328
4,2,Albacete,5,Alborea,663,338,325
...,...,...,...,...,...,...,...
8126,50,Zaragoza,282,"Vilueña, La",65,33,32
8127,50,Zaragoza,295,Vistabella,52,31,21
8128,50,Zaragoza,296,"Zaida, La",435,222,213
8129,50,Zaragoza,297,Zaragoza,673010,321957,351053


Cambio el tipo de dato de algunas de las columnas a string para poder modificar código postal y código municipio ya que se ha prescindido de los ceros del inicio de dichos códigos

In [35]:
poblacion_municipios["CPRO"] = poblacion_municipios["CPRO"].astype(str)
poblacion_municipios["CMUN"] = poblacion_municipios["CMUN"].astype(str)

for index, row in poblacion_municipios.iterrows():
    if len(row["CPRO"]) == 1:
        poblacion_municipios.at[index, "CPRO"] = "0" + row["CPRO"]

for index, row in poblacion_municipios.iterrows():
    if len(row["CMUN"]) == 1:
        poblacion_municipios.at[index, "CMUN"] = "00" + row["CMUN"]
    elif len(row["CMUN"]) == 2:
        poblacion_municipios.at[index, "CMUN"] = "0" + row["CMUN"]

 Concateno la columna "CPRO" y "CMUN" para obtener el código de municipio de cada registro para poder crucar los datos con los de la tabla de códigos postales

In [36]:
poblacion_municipios['cod_municipio'] = poblacion_municipios["CPRO"].str.cat(poblacion_municipios["CMUN"])

In [37]:
poblacion_municipios.head(3)

,CPRO,PROVINCIA,CMUN,NOMBRE,POB22,HOMBRES,MUJERES,cod_municipio
0,02,Albacete,001,Abengibre,739,363,376,02001
1,02,Albacete,002,Alatoz,496,276,220,02002
2,02,Albacete,003,Albacete,172357,84230,88127,02003


In [38]:
cod_postales.head(3)

,codigo_postal,municipio_id,municipio_nombre,codigo_unidad_poblacional,entidad_singular_nombre,nucleo_nombre
0,26120,43110,"Pobla de Massaluca, La",1799,Pobla de Massaluca (La),*Diseminado*
1,43783,49064,Fariza,5699,Palazuelo de Sayago,*Diseminado*
2,49213,26005,Albelda de Iregua,1799,Albelda de Iregua,*Diseminado*


Uno ambas tablas teniendo como columna común el código de municipio

In [40]:
df_poblacion_cod_postal = pd.merge(poblacion_municipios, cod_postales, how= "left", left_on= "cod_municipio", right_on = "municipio_id")

Me quedo únicamente con las columnas que necesito

In [43]:
df_municipios_habitantes_cod_postal = df_poblacion_cod_postal.loc[:,["PROVINCIA","POB22","codigo_postal","municipio_nombre"]]
df_municipios_habitantes_cod_postal

,PROVINCIA,POB22,codigo_postal,municipio_nombre
0,Albacete,739,02250,Abengibre
1,Albacete,496,02152,Alatoz
2,Albacete,496,02152,Alatoz
3,Albacete,172357,02001,Albacete
4,Albacete,172357,02002,Albacete
...,...,...,...,...
77622,Zaragoza,8632,50800,Zuera
77623,Zaragoza,8632,50800,Zuera
77624,Zaragoza,8632,50810,Zuera
77625,Zaragoza,8632,50810,Zuera


Compruebo que existe duplicidad en los códigos postales en la tabla resultante. Son localidades que tienen varios códigos postales o pequeñas localidades que compqarten código postal. en el primer caso puedo presindir de los códigos postales repetidos en el segundo caso no es así, lo resolveré más adelante.

In [44]:
df_municipios_habitantes_cod_postal[df_municipios_habitantes_cod_postal["codigo_postal"]  == "26213"]

,PROVINCIA,POB22,codigo_postal,municipio_nombre
68044,"Rioja, La",104,26213,Herramélluri
68045,"Rioja, La",104,26213,Herramélluri
68067,"Rioja, La",238,26213,Leiva
68122,"Rioja, La",79,26213,Ochánduri
68224,"Rioja, La",114,26213,Tormantos


Cambio el tipo de dato de la columna cod_postal de la tabla de los festivales y corrijo errores de los códigos postales (existen algunos registros con formato erróneo)

In [42]:
df_fest["cod_postal"] = df_fest["cod_postal"].astype(str)

for index, row in df_fest.iterrows():
    df_fest.at[index, "cod_postal"] = row["cod_postal"].split(".")[0]

for index, row in df_fest.iterrows():
    if len(row["cod_postal"]) == 4:
        df_fest.at[index, "cod_postal"] = "0" + row["cod_postal"]

In [45]:
df_fest.head(5)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito
0,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
1,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
2,Locos por la Música 2023 (Fuengirola),Marenostrum Fuengirola,2023-07-29,2023-07-29,1,29640,36.524777,-4.628699,Málaga,Fuengirola,"['Melón Diesel', 'No Me Pises Que Llevo Chancl...","['rock', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
3,A Summer Story 2023,Ciudad del Rock,2023-06-23,2023-06-24,2,28500,40.259650,-3.405805,Madrid,Arganda del Rey,"['Armin van Buuren', 'Don Diablo', 'DJ Nano', ...","['electronic', 'techno', 'house', 'dance', 'EDM']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False
4,Love To Rock 2023 (Valencia),La Marina de Valencia,2023-10-13,2023-10-14,2,46001,39.461771,-0.319321,Valencia,Valencia,"['Quique González', 'Cariño', 'Joe Crepúsculo'...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False


Para resolver el problema de duplicidad de códigos postales, crearé dos dataframes diferenciados. Uno unido por la columna código postal y otro unido por la columna municipio nombre

In [48]:
df_festivales_poblacion = pd.merge(df_fest, df_municipios_habitantes_cod_postal, how= "left", left_on= "cod_postal", right_on = "codigo_postal")
df_festivales_poblacion_001 = pd.merge(df_fest, df_municipios_habitantes_cod_postal, how= "left", left_on= "localidad", right_on = "municipio_nombre")

Como ejemplo para saber que los registros de los festivales se han cruzado bien con los datos de nº de habitantes, tomaré el registro del festival OchanduRock 2023 (festival de localidad más pequeña ya que compartía código postal con otras localidades de La Rioja)

In [52]:
df_festivales_poblacion_001[df_festivales_poblacion_001["cod_postal"]  == "26213"]

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,PROVINCIA,POB22,codigo_postal,municipio_nombre
15892,OchanduRock 2023,Pabellón de l@s Jóvenes,2023-08-19,2023-08-19,1,26213,42.525649,-3.003116,La Rioja,Ochánduri,"['Bull Brigade', 'Radiocrimen', 'Karne Cruda',...","['rock', 'metal', 'punk', 'hardcore', 'ska']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,True,"Rioja, La",79.0,26213,Ochánduri


In [50]:
df_festivales_poblacion.head(5)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,PROVINCIA,POB22,codigo_postal,municipio_nombre
0,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,Girona,32517,17190,Salt
1,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,Girona,32517,17190,Salt
2,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,"Palmas, Las",16924,35570,Yaiza
3,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,"Palmas, Las",16924,35570,Yaiza
4,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,"Palmas, Las",16924,35570,Yaiza


In [53]:
df_festivales_poblacion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1854 entries, 0 to 1853
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   festival          1854 non-null   object 
 1   lugar             1854 non-null   object 
 2   fecha_inicio      1854 non-null   object 
 3   fecha_fin         1854 non-null   object 
 4   duracion          1854 non-null   int64  
 5   cod_postal        1854 non-null   object 
 6   latitud           1854 non-null   float64
 7   longitud          1854 non-null   float64
 8   provincia         1854 non-null   object 
 9   localidad         1854 non-null   object 
 10  artistas          1854 non-null   object 
 11  estilos           1854 non-null   object 
 12  enlace_foto       1696 non-null   object 
 13  cartel_peq        1696 non-null   object 
 14  gratuito          1696 non-null   object 
 15  PROVINCIA         1854 non-null   object 
 16  POB22             1854 non-null   int64  


In [51]:
df_festivales_poblacion_001.head(5)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,PROVINCIA,POB22,codigo_postal,municipio_nombre
0,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,Girona,32517.0,17190,Salt
1,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,Girona,32517.0,17190,Salt
2,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,NaN,NaN,NaN,NaN
3,Locos por la Música 2023 (Fuengirola),Marenostrum Fuengirola,2023-07-29,2023-07-29,1,29640,36.524777,-4.628699,Málaga,Fuengirola,"['Melón Diesel', 'No Me Pises Que Llevo Chancl...","['rock', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,Málaga,83226.0,29640,Fuengirola
4,A Summer Story 2023,Ciudad del Rock,2023-06-23,2023-06-24,2,28500,40.259650,-3.405805,Madrid,Arganda del Rey,"['Armin van Buuren', 'Don Diablo', 'DJ Nano', ...","['electronic', 'techno', 'house', 'dance', 'EDM']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,Madrid,57553.0,28500,Arganda del Rey


In [54]:
df_festivales_poblacion_001.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16519 entries, 0 to 16518
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   festival          16519 non-null  object 
 1   lugar             16519 non-null  object 
 2   fecha_inicio      16519 non-null  object 
 3   fecha_fin         16519 non-null  object 
 4   duracion          16519 non-null  int64  
 5   cod_postal        16519 non-null  object 
 6   latitud           16519 non-null  float64
 7   longitud          16519 non-null  float64
 8   provincia         16519 non-null  object 
 9   localidad         16519 non-null  object 
 10  artistas          16519 non-null  object 
 11  estilos           16519 non-null  object 
 12  enlace_foto       15389 non-null  object 
 13  cartel_peq        15389 non-null  object 
 14  gratuito          15389 non-null  object 
 15  PROVINCIA         16434 non-null  object 
 16  POB22             16434 non-null  float6

Elimino los duplicados de ambos dataframes

In [56]:
df_festivales_poblacion = df_festivales_poblacion.drop_duplicates(subset= "festival")
df_festivales_poblacion_001 = df_festivales_poblacion_001.drop_duplicates(subset= "festival")

Del segundo dataframe limpio de columnas que están ya presentes en el primer dataframe, la idea será finalmente unirlos los dos conservando los datos de habitantes reales de cada registro

In [57]:
df_festivales_poblacion_001 = df_festivales_poblacion_001.loc[:,"festival":"POB22"]
df_festivales_poblacion_001 = df_festivales_poblacion_001.drop(columns= "PROVINCIA")

In [61]:
df_festivales_poblacion.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 0 to 1852
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   festival          414 non-null    object 
 1   lugar             414 non-null    object 
 2   fecha_inicio      414 non-null    object 
 3   fecha_fin         414 non-null    object 
 4   duracion          414 non-null    int64  
 5   cod_postal        414 non-null    object 
 6   latitud           414 non-null    float64
 7   longitud          414 non-null    float64
 8   provincia         414 non-null    object 
 9   localidad         414 non-null    object 
 10  artistas          414 non-null    object 
 11  estilos           414 non-null    object 
 12  enlace_foto       373 non-null    object 
 13  cartel_peq        373 non-null    object 
 14  gratuito          373 non-null    object 
 15  PROVINCIA         414 non-null    object 
 16  POB22             414 non-null    int64  
 

El dataframe resultante con cruzo de nuevo con el del nº de habitantes y código postal 

In [63]:
df_festivales_poblacion_002 = pd.merge(df_festivales_poblacion_001, df_municipios_habitantes_cod_postal, how= "left", left_on= "cod_postal", right_on = "codigo_postal")

In [64]:
df_festivales_poblacion_002.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1854 entries, 0 to 1853
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   festival          1854 non-null   object 
 1   lugar             1854 non-null   object 
 2   fecha_inicio      1854 non-null   object 
 3   fecha_fin         1854 non-null   object 
 4   duracion          1854 non-null   int64  
 5   cod_postal        1854 non-null   object 
 6   latitud           1854 non-null   float64
 7   longitud          1854 non-null   float64
 8   provincia         1854 non-null   object 
 9   localidad         1854 non-null   object 
 10  artistas          1854 non-null   object 
 11  estilos           1854 non-null   object 
 12  enlace_foto       1696 non-null   object 
 13  cartel_peq        1696 non-null   object 
 14  gratuito          1696 non-null   object 
 15  POB22_x           1325 non-null   float64
 16  PROVINCIA         1854 non-null   object 


Borro duplicados teniendo como referencia la columna del festival

In [65]:
df_festivales_poblacion_002 = df_festivales_poblacion_002.drop_duplicates(subset= "festival")

Consulto el registro del pueblo de La Rioja para saber que columna de POB es la que respeta los habitantes reales, y veo que el la POB22_y

In [66]:
df_festivales_poblacion_002[df_festivales_poblacion_002["cod_postal"]  == "26213"]

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,POB22_x,PROVINCIA,POB22_y,codigo_postal,municipio_nombre
1757,OchanduRock 2023,Pabellón de l@s Jóvenes,2023-08-19,2023-08-19,1,26213,42.525649,-3.003116,La Rioja,Ochánduri,"['Bull Brigade', 'Radiocrimen', 'Karne Cruda',...","['rock', 'metal', 'punk', 'hardcore', 'ska']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,True,79.0,"Rioja, La",104,26213,Herramélluri


Cambioel tipo de dato de dicha columna a string

In [67]:
df_festivales_poblacion_002["POB22_x"] = df_festivales_poblacion_002["POB22_x"].astype(str)

Compruebo que tengo todos los registros del principio

In [68]:
df_festivales_poblacion_002.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 0 to 1852
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   festival          414 non-null    object 
 1   lugar             414 non-null    object 
 2   fecha_inicio      414 non-null    object 
 3   fecha_fin         414 non-null    object 
 4   duracion          414 non-null    int64  
 5   cod_postal        414 non-null    object 
 6   latitud           414 non-null    float64
 7   longitud          414 non-null    float64
 8   provincia         414 non-null    object 
 9   localidad         414 non-null    object 
 10  artistas          414 non-null    object 
 11  estilos           414 non-null    object 
 12  enlace_foto       373 non-null    object 
 13  cartel_peq        373 non-null    object 
 14  gratuito          373 non-null    object 
 15  POB22_x           414 non-null    object 
 16  PROVINCIA         414 non-null    object 
 

Traslado los datos de la columna POB22_y a los nulos de la columna POB22_x y así poder luego prescindir de ella

In [69]:
for index, row in df_festivales_poblacion_002.iterrows():
    if row["POB22_x"] == "nan":
        df_festivales_poblacion_002.at[index, "POB22_x"] = row["POB22_y"]

Cambio el tipo de dato de la columna y corrijo errores en cuanto a formato

In [70]:
df_festivales_poblacion_002["POB22_x"] = df_festivales_poblacion_002["POB22_x"].astype(str)

for index, row in df_festivales_poblacion_002.iterrows():
    df_festivales_poblacion_002.at[index, "POB22_x"] = row["POB22_x"].split(".")[0]

In [71]:
df_festivales_poblacion_002.head(5)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,POB22_x,PROVINCIA,POB22_y,codigo_postal,municipio_nombre
0,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,32517,Girona,32517,17190,Salt
2,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,16924,"Palmas, Las",16924,35570,Yaiza
18,Locos por la Música 2023 (Fuengirola),Marenostrum Fuengirola,2023-07-29,2023-07-29,1,29640,36.524777,-4.628699,Málaga,Fuengirola,"['Melón Diesel', 'No Me Pises Que Llevo Chancl...","['rock', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,Málaga,83226,29640,Fuengirola
19,A Summer Story 2023,Ciudad del Rock,2023-06-23,2023-06-24,2,28500,40.259650,-3.405805,Madrid,Arganda del Rey,"['Armin van Buuren', 'Don Diablo', 'DJ Nano', ...","['electronic', 'techno', 'house', 'dance', 'EDM']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,57553,Madrid,57553,28500,Arganda del Rey
25,Love To Rock 2023 (Valencia),La Marina de Valencia,2023-10-13,2023-10-14,2,46001,39.461771,-0.319321,Valencia,Valencia,"['Quique González', 'Cariño', 'Joe Crepúsculo'...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,792492,Valencia/València,792492,46001,Valencia


In [72]:
df_festivales_poblacion_002[df_festivales_poblacion_002["cod_postal"]  == "26213"]

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,POB22_x,PROVINCIA,POB22_y,codigo_postal,municipio_nombre
1757,OchanduRock 2023,Pabellón de l@s Jóvenes,2023-08-19,2023-08-19,1,26213,42.525649,-3.003116,La Rioja,Ochánduri,"['Bull Brigade', 'Radiocrimen', 'Karne Cruda',...","['rock', 'metal', 'punk', 'hardcore', 'ska']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,True,79,"Rioja, La",104,26213,Herramélluri


Rellenados los valores nulos por los valores de la columna POB22_y, prescindo de las columnas que ya no necesito y renombro la columna del nº de habitantes

In [74]:
df_festivales_poblacion_002 = df_festivales_poblacion_002.loc[:,"festival":"POB22_x"]
df_festivales_poblacion_002 = df_festivales_poblacion_002.rename(columns={'POB22_x': 'habitantes'})

Cambio el tipo de dato de la columna habitantes a entero para poder operar más adelante con este tipo de dato

In [76]:
df_festivales_poblacion_002["habitantes"] = df_festivales_poblacion_002["habitantes"].astype("int64")

In [77]:
df_festivales_poblacion_002.head(5)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,habitantes
0,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,32517
2,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,16924
18,Locos por la Música 2023 (Fuengirola),Marenostrum Fuengirola,2023-07-29,2023-07-29,1,29640,36.524777,-4.628699,Málaga,Fuengirola,"['Melón Diesel', 'No Me Pises Que Llevo Chancl...","['rock', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226
19,A Summer Story 2023,Ciudad del Rock,2023-06-23,2023-06-24,2,28500,40.259650,-3.405805,Madrid,Arganda del Rey,"['Armin van Buuren', 'Don Diablo', 'DJ Nano', ...","['electronic', 'techno', 'house', 'dance', 'EDM']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,57553
25,Love To Rock 2023 (Valencia),La Marina de Valencia,2023-10-13,2023-10-14,2,46001,39.461771,-0.319321,Valencia,Valencia,"['Quique González', 'Cariño', 'Joe Crepúsculo'...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,792492


In [78]:
df_festivales_poblacion_002.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 0 to 1852
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   festival      414 non-null    object 
 1   lugar         414 non-null    object 
 2   fecha_inicio  414 non-null    object 
 3   fecha_fin     414 non-null    object 
 4   duracion      414 non-null    int64  
 5   cod_postal    414 non-null    object 
 6   latitud       414 non-null    float64
 7   longitud      414 non-null    float64
 8   provincia     414 non-null    object 
 9   localidad     414 non-null    object 
 10  artistas      414 non-null    object 
 11  estilos       414 non-null    object 
 12  enlace_foto   373 non-null    object 
 13  cartel_peq    373 non-null    object 
 14  gratuito      373 non-null    object 
 15  habitantes    414 non-null    int64  
dtypes: float64(2), int64(2), object(12)
memory usage: 71.1+ KB


In [79]:
df_festivales_poblacion_002["habitantes"].min() # Dato del pueblo de La Rioja

79

In [80]:
df_festivales_poblacion_002["habitantes"].max() # Dato de Madrid

3280782

Creo una función para discretizar la columna de los habitantes y agruparla por tamaño de población

In [81]:
def intervalo(habitantes):
    if habitantes <= 5000:
        return '0 - 5K'
    elif habitantes >= 5000 and habitantes <= 10000:
        return '5K - 10K'
    elif habitantes > 10000 and habitantes <= 50000:
        return '10K - 50K'
    elif habitantes > 50000 and habitantes <= 100000:
        return '50K - 100K'
    elif habitantes > 100000 and habitantes <= 500000:
        return '100K - 500K'
    elif habitantes > 500000 and habitantes <= 1000000:
        return '500K - 1M'
    elif habitantes > 1000000 and habitantes <= 2000000:
        return '1M - 2M'
    elif habitantes > 2000000:
        return '2M - 3.2M'

In [82]:
df_festivales_poblacion_002['tamaño_localidad'] = df_festivales_poblacion_002['habitantes'].apply(lambda x: intervalo(x))

In [83]:
df_festivales_poblacion_002.head(5)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,habitantes,tamaño_localidad
0,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,32517,10K - 50K
2,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,16924,10K - 50K
18,Locos por la Música 2023 (Fuengirola),Marenostrum Fuengirola,2023-07-29,2023-07-29,1,29640,36.524777,-4.628699,Málaga,Fuengirola,"['Melón Diesel', 'No Me Pises Que Llevo Chancl...","['rock', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K
19,A Summer Story 2023,Ciudad del Rock,2023-06-23,2023-06-24,2,28500,40.259650,-3.405805,Madrid,Arganda del Rey,"['Armin van Buuren', 'Don Diablo', 'DJ Nano', ...","['electronic', 'techno', 'house', 'dance', 'EDM']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,57553,50K - 100K
25,Love To Rock 2023 (Valencia),La Marina de Valencia,2023-10-13,2023-10-14,2,46001,39.461771,-0.319321,Valencia,Valencia,"['Quique González', 'Cariño', 'Joe Crepúsculo'...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,792492,500K - 1M


Cambio el tipo de dato de las columnas de fecha

In [84]:
df_festivales_poblacion_002["fecha_inicio"] = pd.to_datetime(df_festivales_poblacion_002["fecha_inicio"])
df_festivales_poblacion_002["fecha_fin"] = pd.to_datetime(df_festivales_poblacion_002["fecha_fin"])

Creo una columna con el nombre del mes y cambio el tipo de dato de datetime a string

In [87]:
for index, row in df_festivales_poblacion_002.iterrows():
    df_festivales_poblacion_002.at[index, 'mes'] = row['fecha_inicio'].month_name()

df_festivales_poblacion_002["mes"] = df_festivales_poblacion_002["mes"].astype(str)

In [88]:
df_festivales_poblacion_002.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 414 entries, 0 to 1852
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   festival          414 non-null    object        
 1   lugar             414 non-null    object        
 2   fecha_inicio      414 non-null    datetime64[ns]
 3   fecha_fin         414 non-null    datetime64[ns]
 4   duracion          414 non-null    int64         
 5   cod_postal        414 non-null    object        
 6   latitud           414 non-null    float64       
 7   longitud          414 non-null    float64       
 8   provincia         414 non-null    object        
 9   localidad         414 non-null    object        
 10  artistas          414 non-null    object        
 11  estilos           414 non-null    object        
 12  enlace_foto       373 non-null    object        
 13  cartel_peq        373 non-null    object        
 14  gratuito          373 non

Creo una función para que se muestre el nombre de los meses en castellano

In [90]:
def mes_castellano(mes):
    if mes == "January":
        return 'Enero'
    elif mes == "February":
        return 'Febrero'
    elif mes == "March":
        return 'Marzo'
    elif mes == "April":
        return 'Abril'
    elif mes == "May":
        return 'Mayo'
    elif mes == "June":
        return 'Junio'
    elif mes == "July":
        return 'Julio'
    elif mes == "August":
        return 'Agosto'
    elif mes == "September":
        return 'Septiembre'
    elif mes == "October":
        return 'Octubre'
    elif mes == "November":
        return 'Noviembre'
    elif mes == "December":
        return 'Diciembre'

In [91]:
df_festivales_poblacion_002['mes'] = df_festivales_poblacion_002['mes'].apply(lambda x: mes_castellano(x))
df_festivales_poblacion_002.head(5)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,habitantes,tamaño_localidad,mes
0,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,32517,10K - 50K,Mayo
2,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,16924,10K - 50K,Junio
18,Locos por la Música 2023 (Fuengirola),Marenostrum Fuengirola,2023-07-29,2023-07-29,1,29640,36.524777,-4.628699,Málaga,Fuengirola,"['Melón Diesel', 'No Me Pises Que Llevo Chancl...","['rock', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K,Julio
19,A Summer Story 2023,Ciudad del Rock,2023-06-23,2023-06-24,2,28500,40.259650,-3.405805,Madrid,Arganda del Rey,"['Armin van Buuren', 'Don Diablo', 'DJ Nano', ...","['electronic', 'techno', 'house', 'dance', 'EDM']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,57553,50K - 100K,Junio
25,Love To Rock 2023 (Valencia),La Marina de Valencia,2023-10-13,2023-10-14,2,46001,39.461771,-0.319321,Valencia,Valencia,"['Quique González', 'Cariño', 'Joe Crepúsculo'...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,792492,500K - 1M,Octubre


Listado de provincias costeras

In [93]:
provincias_con_costa = ["A Coruña", "Lugo", "Pontevedra", "Cádiz", "Huelva", 
                        "Granada", "Málaga", "Almería", "Murcia", "Girona", 
                        "Tarragona", "Barcelona", "Castellón", "Valencia", 
                        "Alicante", "Cantabria", "Asturias", "Guipuzcoa", 
                        "Vizcaya", "Las Palmas", "Santa Cruz de Tenerife", 
                        "Islas Baleares"]
len(provincias_con_costa)

22

Creo una nueva columna donde indique si la provincia es costera o no

In [94]:
df_festivales_poblacion_002["costa"]= df_festivales_poblacion_002['provincia'].apply(lambda x: True if x in provincias_con_costa else False)
df_festivales_poblacion_002.head(5)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,habitantes,tamaño_localidad,mes,costa
0,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,32517,10K - 50K,Mayo,True
2,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,16924,10K - 50K,Junio,True
18,Locos por la Música 2023 (Fuengirola),Marenostrum Fuengirola,2023-07-29,2023-07-29,1,29640,36.524777,-4.628699,Málaga,Fuengirola,"['Melón Diesel', 'No Me Pises Que Llevo Chancl...","['rock', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K,Julio,True
19,A Summer Story 2023,Ciudad del Rock,2023-06-23,2023-06-24,2,28500,40.259650,-3.405805,Madrid,Arganda del Rey,"['Armin van Buuren', 'Don Diablo', 'DJ Nano', ...","['electronic', 'techno', 'house', 'dance', 'EDM']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,57553,50K - 100K,Junio,False
25,Love To Rock 2023 (Valencia),La Marina de Valencia,2023-10-13,2023-10-14,2,46001,39.461771,-0.319321,Valencia,Valencia,"['Quique González', 'Cariño', 'Joe Crepúsculo'...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,792492,500K - 1M,Octubre,True


Lista con la comunidades autónomas y sus respectivas províncias y una función que asigne la comunidad según la província. Creo una nueva columna con esta información

In [95]:
comunidades_autonomas = {
    'Andalucía': ['Almería', 'Cádiz', 'Córdoba', 'Granada', 'Huelva', 'Jaén', 'Málaga', 'Sevilla'],
    'Aragón': ['Huesca', 'Teruel', 'Zaragoza'],
    'Asturias': ['Asturias'],
    'Islas Baleares': ['Islas Baleares'],
    'Canarias': ['Las Palmas', 'Santa Cruz de Tenerife'],
    'Cantabria': ['Cantabria'],
    'Castilla y León': ['Ávila', 'Burgos', 'León', 'Palencia', 'Salamanca', 'Segovia', 'Soria', 'Valladolid', 'Zamora'],
    'Castilla-La Mancha': ['Albacete', 'Ciudad Real', 'Cuenca', 'Guadalajara', 'Toledo'],
    'Cataluña': ['Barcelona', 'Girona', 'Lleida', 'Tarragona'],
    'Extremadura': ['Badajoz', 'Cáceres'],
    'Galicia': ['A Coruña', 'Lugo', 'Ourense', 'Pontevedra'],
    'Madrid': ['Madrid'],
    'Murcia': ['Murcia'],
    'Navarra': ['Navarra'],
    'La Rioja': ['La Rioja'],
    'País Vasco': ['Álava', 'Guipuzcoa', 'Vizcaya'],
    'Comunidad Valenciana': ['Alicante', 'Castellón', 'Valencia'],
    'Ceuta': ['Ceuta'],
    'Melilla': ['Melilla']
}

def establecer_comunidad_autonoma(provincia):
    for comunidad, provincias in comunidades_autonomas.items():
        if provincia in provincias:
            return comunidad
    return None

df_festivales_poblacion_002['comunidad_autonoma'] = df_festivales_poblacion_002['provincia'].apply(establecer_comunidad_autonoma)

Conteo de la cantidad de festivales por Comunidad Autónoma

In [96]:
df_festivales_poblacion_002["comunidad_autonoma"].value_counts()

Andalucía               79
Comunidad Valenciana    48
Cataluña                44
Castilla y León         40
Madrid                  37
Galicia                 26
País Vasco              20
Castilla-La Mancha      18
Extremadura             16
Murcia                  15
Canarias                13
Asturias                13
Aragón                  12
Islas Baleares          12
Cantabria               10
La Rioja                 6
Navarra                  5
Name: comunidad_autonoma, dtype: int64

In [97]:
df_festivales_poblacion_002.head(5)

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,habitantes,tamaño_localidad,mes,costa,comunidad_autonoma
0,No Future Fest 2023 (Vol.5),Sala la Mirona,2023-05-20,2023-05-20,1,17190,41.968265,2.773426,Girona,Salt,"['The Exploited', 'Discharge', 'The Profession...","['rock', 'punk', 'punk rock']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,32517,10K - 50K,Mayo,True,Cataluña
2,Sonidos Líquidos 2023 - Malvasía Volcánica Wee...,Bodegas La Geria,2023-06-09,2023-06-10,2,35570,28.969307,-13.714773,Las Palmas,La Geria,"['Depedro', 'Arde Bogotá', 'Drûpe']","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,16924,10K - 50K,Junio,True,Canarias
18,Locos por la Música 2023 (Fuengirola),Marenostrum Fuengirola,2023-07-29,2023-07-29,1,29640,36.524777,-4.628699,Málaga,Fuengirola,"['Melón Diesel', 'No Me Pises Que Llevo Chancl...","['rock', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K,Julio,True,Andalucía
19,A Summer Story 2023,Ciudad del Rock,2023-06-23,2023-06-24,2,28500,40.259650,-3.405805,Madrid,Arganda del Rey,"['Armin van Buuren', 'Don Diablo', 'DJ Nano', ...","['electronic', 'techno', 'house', 'dance', 'EDM']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,57553,50K - 100K,Junio,False,Madrid
25,Love To Rock 2023 (Valencia),La Marina de Valencia,2023-10-13,2023-10-14,2,46001,39.461771,-0.319321,Valencia,Valencia,"['Quique González', 'Cariño', 'Joe Crepúsculo'...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,792492,500K - 1M,Octubre,True,Comunidad Valenciana


Conteo de la cantidad de festivales por Comunidad Autónoma

In [98]:
df_festivales_poblacion_002.groupby("comunidad_autonoma")["provincia"].count()

comunidad_autonoma
Andalucía               79
Aragón                  12
Asturias                13
Canarias                13
Cantabria               10
Castilla y León         40
Castilla-La Mancha      18
Cataluña                44
Comunidad Valenciana    48
Extremadura             16
Galicia                 26
Islas Baleares          12
La Rioja                 6
Madrid                  37
Murcia                  15
Navarra                  5
País Vasco              20
Name: provincia, dtype: int64

Veo que entre las 20 primeras localidades, únicamente hay 5 de ellas que no son capitales de província (Fuengirola, Benidorm, Arganda del Rey, Villena y Benicássim)

In [99]:
df_festivales_poblacion_002.groupby("localidad")[["festival"]].count().reset_index().sort_values(ascending=False, by="festival").head(20)

,localidad,festival
136,Madrid,29
34,Barcelona,13
234,Valencia,13
149,Murcia,10
97,Fuengirola,8
213,Sevilla,8
39,Benidorm,6
48,Burgos,6
10,Alicante,5
101,Gijón,4


In [101]:
df_festivales_poblacion_002[df_festivales_poblacion_002["localidad"]== "Fuengirola"]

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,habitantes,tamaño_localidad,mes,costa,comunidad_autonoma
18,Locos por la Música 2023 (Fuengirola),Marenostrum Fuengirola,2023-07-29,2023-07-29,1,29640,36.524777,-4.628699,Málaga,Fuengirola,"['Melón Diesel', 'No Me Pises Que Llevo Chancl...","['rock', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K,Julio,True,Andalucía
135,Ultra Beach Costa del Sol 2023,Marenostrum Fuengirola,2023-06-24,2023-06-24,1,29640,36.524777,-4.628699,Málaga,Fuengirola,"['Martin Garrix', 'Wade', 'Vintage Culture']","['electronic', 'dj', 'techno', 'house', 'dance']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K,Junio,True,Andalucía
290,Marenostrum Fuengirola 2023,Marenostrum Fuengirola,2023-05-13,2023-08-19,99,29640,36.525226,-4.627776,Málaga,Fuengirola,"['Robbie Williams', 'Vetusta Morla', 'Marea', ...","['rock', 'pop', 'latin']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K,Mayo,True,Andalucía
533,Boombastic Day Costa del Sol 2023,Marenostrum Fuengirola,2023-08-12,2023-08-12,1,29640,36.524777,-4.628699,Málaga,Fuengirola,['Bizarrap'],"['pop', 'electronic', 'hip-hop', 'urban']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K,Agosto,True,Andalucía
801,Fulanita Fest 2023,Marenostrum Fuengirola,2023-05-13,2023-05-13,1,29640,36.524777,-4.628699,Málaga,Fuengirola,"['Ginebras', 'Ptazeta', 'Ladilla rusa', 'María...","['indie', 'pop', 'fusion', 'electro pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K,Mayo,True,Andalucía
872,Cosquín Rock España 2023,Marenostrum Fuengirola,2023-06-17,2023-06-17,1,29640,36.523588,-4.629278,Málaga,Fuengirola,"['Marea', 'Bocanada', 'Skay y Los Fakires', 'L...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K,Junio,True,Andalucía
1024,Cabaret Festival Fuengirola 2023,Marenostrum Fuengirola,2023-08-01,2023-08-31,31,29640,36.524777,-4.628699,Málaga,Fuengirola,"['El Arrebato', 'Los Morancos']","['pop', 'fusion']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K,Agosto,True,Andalucía
1033,Luna Sur Festival 2023,Marenostrum Fuengirola,2023-08-05,2023-08-05,1,29640,36.523588,-4.629278,Málaga,Fuengirola,"['Vetusta Morla', 'Miss Caffeina', 'Anni B Swe...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,83226,50K - 100K,Agosto,True,Andalucía


In [102]:
df_festivales_poblacion_002[df_festivales_poblacion_002["localidad"]== "Benidorm"]

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,habitantes,tamaño_localidad,mes,costa,comunidad_autonoma
91,Homenaje a la Ruta 90's & 2MIL Benidorm 2023,Auditorio Julio Iglesias,2023-06-10,2023-06-10,1,03501,38.542334,-0.130211,Alicante,Benidorm,['Kate Ryan'],"['electronic', 'dj', 'dance']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,69738,50K - 100K,Junio,True,Comunidad Valenciana
1012,Low Festival 2023,Ciudad Deportiva Guillermo Amor,2023-07-28,2023-07-30,3,03502,38.546691,-0.138434,Alicante,Benidorm,"['Vetusta Morla', 'Placebo', 'Interpol', 'Iván...","['rock', 'indie', 'pop', 'electronic']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,69738,50K - 100K,Julio,True,Comunidad Valenciana
1189,Benidorm Fest 2023,Palacio de Deportes L'Illa de Benidorm,2023-01-31,2023-02-04,5,03502,38.549760,-0.119938,Alicante,Benidorm,"['Agoney', 'Alfred García', 'Alice Wonder', 'B...","['rock', 'indie', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,69738,50K - 100K,Enero,True,Comunidad Valenciana
1266,Reggaeton Beach Festival (Benidorm) 2023,Estadio Municipal Guillermo Amor,2023-07-08,2023-07-09,2,03502,38.547410,-0.138016,Alicante,Benidorm,['Jowell & Randy'],"['urban', 'latin', 'reggaeton']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,69738,50K - 100K,Julio,True,Comunidad Valenciana
1434,Beach Riot Fest 2023,Discoteca Ku,2023-05-05,2023-05-06,2,03501,38.546140,-0.117726,Alicante,Benidorm,"['Bones Of Minerva', 'Crisix', 'El Altar del H...","['rock', 'metal', 'punk']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,69738,50K - 100K,Mayo,True,Comunidad Valenciana
1789,Iberia Festival 2023,Auditorio Julio Iglesias,2023-09-29,2023-09-30,2,03501,38.542232,-0.130334,Alicante,Benidorm,sin artistas,"['rock', 'pop']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,69738,50K - 100K,Septiembre,True,Comunidad Valenciana


In [103]:
df_festivales_poblacion_002[df_festivales_poblacion_002["localidad"]== "Arganda del Rey"]

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,habitantes,tamaño_localidad,mes,costa,comunidad_autonoma
19,A Summer Story 2023,Ciudad del Rock,2023-06-23,2023-06-24,2,28500,40.259650,-3.405805,Madrid,Arganda del Rey,"['Armin van Buuren', 'Don Diablo', 'DJ Nano', ...","['electronic', 'techno', 'house', 'dance', 'EDM']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,57553,50K - 100K,Junio,False,Madrid
340,Primavera Sound Madrid 2023,Ciudad del Rock,2023-06-08,2023-06-10,3,28500,40.260288,-3.405719,Madrid,Arganda del Rey,"['Rosalía', 'Depeche Mode', 'Blur', 'FKA Twigs...","['rock', 'indie', 'pop', 'electronic', 'altern...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,57553,50K - 100K,Junio,False,Madrid
1234,Puro Latino Fest Madrid 2023,Ciudad del Rock,2023-06-30,2023-07-01,2,28500,40.259655,-3.405773,Madrid,Arganda del Rey,"['J Balvin', 'Dellafuente', 'Eladio Carrión', ...","['urban', 'latin', 'reggaeton']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,57553,50K - 100K,Junio,False,Madrid


In [104]:
df_festivales_poblacion_002[df_festivales_poblacion_002["localidad"]== "Villena"]

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,habitantes,tamaño_localidad,mes,costa,comunidad_autonoma
585,Rabolagartija Festival 2023,Polideportivo Municipal de Villena,2023-08-17,2023-08-19,3,03400,38.631858,-0.846634,Alicante,Villena,"['Desakato', 'La M.O.D.A.', 'La Pegatina', 'Ma...","['rock', 'hip-hop', 'fusion', 'funk', 'flamenco']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,33969,10K - 50K,Agosto,True,Comunidad Valenciana
1730,Leyendas del Rock 2023,Polideportivo Municipal,2023-08-09,2023-08-12,4,03400,38.631912,-0.847385,Alicante,Villena,"['Megadeth', 'Arch Enemy', 'Bullet for My Vale...","['rock', 'metal', 'heavy metal', 'hard rock', ...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,33969,10K - 50K,Agosto,True,Comunidad Valenciana
1820,F*ck The Censorship Festival 2023,Plaza de Toros,2023-01-28,2023-01-28,1,03400,38.639604,-0.868354,Alicante,Villena,"['Soziedad Alkoholika', 'Lendakaris Muertos', ...","['rock', 'metal', 'punk', 'punk rock', 'rock u...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,33969,10K - 50K,Enero,True,Comunidad Valenciana


In [100]:
df_festivales_poblacion_002[df_festivales_poblacion_002["localidad"]== "Benicássim"]

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,artistas,estilos,enlace_foto,cartel_peq,gratuito,habitantes,tamaño_localidad,mes,costa,comunidad_autonoma
144,FIB 2023,Benicàssim,2023-07-13,2023-07-16,4,12560,40.048068,0.047872,Castellón,Benicássim,"['Franz Ferdinand', 'Bastille', 'The Offspring...","['rock', 'indie', 'pop', 'electronic', 'altern...",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,19507,10K - 50K,Julio,True,Comunidad Valenciana
569,Rototom Sunsplash 2023,Recinto de Festivales (Benicàssim),2023-08-16,2023-08-22,7,12560,40.048249,0.048687,Castellón,Benicássim,"['UB40', 'Trueno', 'Gentleman', 'Zoo', 'Adala'...","['reggae', 'ska', 'dub', 'dancehall']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,19507,10K - 50K,Agosto,True,Comunidad Valenciana
734,Sansan Festival 2023,Ciudad de los Festivales,2023-04-06,2023-04-08,3,12560,40.048257,0.048913,Castellón,Benicássim,"['Phoenix', 'Leiva', 'Love of Lesbian', 'Alizz...","['indie', 'pop', 'indie pop', 'dj']",https://fanmusicfest.b-cdn.net/sites/default/f...,https://fanmusicfest.b-cdn.net/sites/default/f...,False,19507,10K - 50K,Abril,True,Comunidad Valenciana


Conteo de la cantidad de festivales según tamaño de localidad

In [106]:
df_festivales_poblacion_002["tamaño_localidad"].value_counts()

10K - 50K      104
100K - 500K     90
50K - 100K      62
0 - 5K          53
5K - 10K        35
2M - 3.2M       30
500K - 1M       27
1M - 2M         13
Name: tamaño_localidad, dtype: int64

Creo columnas de electronica, indie, rock, pop para poder filtrar por estilo de música

In [110]:
df_festivales_poblacion_002["electronica"] = df_festivales_poblacion_002["estilos"].apply(lambda x: True if 'electronic' in x.lower() else False)
df_festivales_poblacion_002["indie"] = df_festivales_poblacion_002["estilos"].apply(lambda x: True if 'indie' in x.lower() else False)
df_festivales_poblacion_002["rock"] = df_festivales_poblacion_002["estilos"].apply(lambda x: True if 'rock' in x.lower() else False)
df_festivales_poblacion_002["pop"] = df_festivales_poblacion_002["estilos"].apply(lambda x: True if 'pop' in x.lower() else False)

Según unas características concretas filtro los festivales y veo que únicamente cumplen esas condiciones 6 festivales. Los 6 se concentran en época estival

In [111]:
fest_gratuitos = df_festivales_poblacion_002[df_festivales_poblacion_002["gratuito"] == False]
fest_10k_50k = fest_gratuitos[fest_gratuitos["tamaño_localidad"]== "10K - 50K"]
fest_interior = fest_10k_50k[fest_10k_50k["costa"]== False]
fest_indie = fest_interior[fest_interior["indie"]== True]
fest_pop = fest_indie[fest_indie["pop"]== True]
fest_rock = fest_pop[fest_pop["rock"]== True]
fest_rock

,festival,lugar,fecha_inicio,fecha_fin,duracion,cod_postal,latitud,longitud,provincia,localidad,...,gratuito,habitantes,tamaño_localidad,mes,costa,comunidad_autonoma,electronica,indie,rock,pop
992,Fárdelej Festival Rioja Live 2023,Arnedo,2023-07-14,2023-07-15,2,26580,42.225735,-2.102074,La Rioja,Arnedo,...,False,15017,10K - 50K,Julio,False,La Rioja,False,True,True,True
1036,Sonorama Ribera 2023,Recinto Ferial,2023-08-09,2023-08-13,5,09400,41.670567,-3.700037,Burgos,Aranda de Duero,...,False,33172,10K - 50K,Agosto,False,Castilla y León,False,True,True,True
1092,Contempopránea 2023,Baluartes de San Blás,2023-08-24,2023-08-26,3,06100,38.683839,-7.096774,Badajoz,Olivenza,...,False,11832,10K - 50K,Agosto,False,Extremadura,False,True,True,True
1098,Ebrovisión 2023,Varios Recintos,2023-08-31,2023-09-03,4,09200,42.693542,-2.914177,Burgos,Miranda de Ebro,...,False,35239,10K - 50K,Agosto,False,Castilla y León,False,True,True,True
1124,Cabaret Festival Latino Sevilla 2023,Recinto Hípico,2023-09-02,2023-09-14,13,41927,37.346978,-6.053660,Sevilla,Mairena del Aljarafe,...,False,47161,10K - 50K,Septiembre,False,Andalucía,False,True,True,True
1126,Cabaret Festival Mairena del Aljarafe 2023,Recinto Hípico,2023-09-02,2023-09-30,29,41927,37.346978,-6.053660,Sevilla,Mairena del Aljarafe,...,False,47161,10K - 50K,Septiembre,False,Andalucía,False,True,True,True


Extraigo de la columna estilos todas las ocurrencias de cada estilos de música (sin agrupar, como etiquetas) para llevar un conteo

In [114]:
estilos_todos = []
for index, row in df_festivales_poblacion_002.iterrows():
    for estilo in row["estilos"].strip("[]").replace("' ","").lstrip().replace("'","").split(","):
        estilos_todos.append(estilo.lstrip().rstrip())

estilos_analisis = pd.DataFrame(estilos_todos)
estilos_analisis = estilos_analisis.rename(columns= {0: "estilos"})
estilos_analisis.value_counts()

estilos            
rock                   188
pop                    174
indie                  132
urban                   74
electronic              73
                      ... 
alt-country              1
krautrock                1
melodic death metal      1
melodic metal            1
video                    1
Length: 90, dtype: int64

Convierto en dataframe para graficar

In [115]:
estilos_analisis = pd.DataFrame(estilos_analisis.value_counts()).reset_index()
estilos_analisis = estilos_analisis.rename(columns={0:"cuenta_estilos"})
estilos_analisis

,estilos,cuenta_estilos
0,rock,188
1,pop,174
2,indie,132
3,urban,74
4,electronic,73
...,...,...
85,alt-country,1
86,krautrock,1
87,melodic death metal,1
88,melodic metal,1


In [117]:
import plotly.express as px

palette = ["#660000","#990033","#5F021F","#8C001A","#FF9000"]
palette_10 = ["#03071e","#370617","#6a040f","#9d0208","#d00000","#dc2f02","#e85d04","#f48c06","#faa307","#ffba08"]
values = estilos_analisis["cuenta_estilos"]

fig = px.bar(estilos_analisis.head(10),
            x='estilos',
            y='cuenta_estilos')

fig.update_traces(
                marker_color=np.tile(palette_10[::-1], len(estilos_analisis["estilos"])),
                text=values,
                textposition='auto',
                textfont=dict(size=16))

fig.update_xaxes(
                tickangle=-45,
                tickfont=dict(size=18))

fig.update_yaxes(
                tickfont=dict(size=18),
                gridcolor='gray')


fig.update_layout(
                    width=1000,
                    height=600,
                    title_text="Top 10 estilos musicales",
                    title_font=dict(size=30),
                    title_x=0.5, title_y=0.95,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    yaxis_title='Cantidad de ocurrencias',
                    xaxis_title='Estilos musicales',
                    yaxis_title_font=dict(size=25),
                    xaxis_title_font=dict(size=25),
                    font=dict(family='Century Gothic'),                           
                    showlegend=False)
                    

fig.add_annotation(x=9.4, y=0, xanchor='right', yanchor='top',
                    text='Fuente: fanmusicfest.com',
                    font=dict(family='Century Gothic', size=16),
                    showarrow=False)  
fig.write_image(r'graficos\grafico_barras_estilos.png')
fig.show()

In [118]:
import plotly.express as px

palette = ["#660000","#990033","#5F021F","#8C001A","#FF9000"]
palette_10 = ["#03071e","#370617","#6a040f","#9d0208","#d00000","#dc2f02","#e85d04","#f48c06","#faa307","#ffba08"]
values = estilos_analisis["cuenta_estilos"]

fig = px.bar(estilos_analisis.head(20),
            x='estilos',
            y='cuenta_estilos')

fig.update_traces(
                marker_color=np.tile(palette_10, len(estilos_analisis["estilos"])),
                text=values,
                textposition='auto',
                textfont=dict(size=16))

fig.update_xaxes(
                tickangle=-45,
                tickfont=dict(size=18))

fig.update_yaxes(
                tickfont=dict(size=18),
                gridcolor='gray')


fig.update_layout(
                    width=1000,
                    height=600,
                    title_text="Top 20 estilos musicales",
                    title_font=dict(size=30),
                    title_x=0.5, title_y=0.95,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    yaxis_title='Cantidad de ocurrencias',
                    xaxis_title='Estilos musicales',
                    yaxis_title_font=dict(size=25),
                    xaxis_title_font=dict(size=25),
                    font=dict(family='Century Gothic'),                           
                    showlegend=False)
                    

fig.add_annotation(x=19.4, y=0, xanchor='right', yanchor='top',
                    text='Fuente: fanmusicfest.com',
                    font=dict(family='Century Gothic', size=12),
                    showarrow=False)  
fig.write_image(r'graficos\grafico_barras_estilos_top20.png')
fig.show()

Extraigo de la columna artistas todas las ocurrencias de grupo (sin agrupar, como etiquetas) para llevar un conteo

In [120]:
artistas_todos = []
for index, row in df_festivales_poblacion_002.iterrows():
    for artista in row["artistas"].strip().strip("[]").replace("'","").split(","):
        artistas_todos.append(artista.lstrip().rstrip())

artistas_analisis = pd.DataFrame(artistas_todos)
artistas_analisis = artistas_analisis.rename(columns={0: 'artistas'})
artistas_analisis

,artistas
0,The Exploited
1,Discharge
2,The Professionals
3,One Way System
4,Subhumans
...,...
4493,The Flaming Shakers
4494,Wax & Boogie
4495,Serious Black
4496,Blast Wave


Creo un dataframe con la cantidad de ocurrencias

In [121]:
artistas_analisis = pd.DataFrame(artistas_analisis.value_counts().reset_index())
artistas_analisis = artistas_analisis.rename(columns={0: 'cuenta_artistas'})
artistas_analisis = artistas_analisis.iloc[1:]
artistas_analisis

,artistas,cuenta_artistas
1,Viva Suecia,23
2,Second,21
3,Arde Bogotá,19
4,Vetusta Morla,18
5,Narco,17
...,...,...
2493,Gonzalo Portugal,1
2494,Gomad! & Monster,1
2495,Golgotha,1
2496,Goalmaker,1


Creo un dataframe únicamente con los 10 artistas mas repetidos y sus escuchas mensuales en Spotify

In [123]:
artistas_top_10 = artistas_analisis.loc[:, ["artistas", "cuenta_artistas"]].head(10)
artistas_top_10["escuchas_mensuales"] = escuchas_spotify = ["652.897", "209.925", "352.128", "1.115.111", "55.234", "218.680", "663.734", "394.266", "1.350.585", "790.461"]
artistas_top_10

,artistas,cuenta_artistas,escuchas_mensuales
1,Viva Suecia,23,652.897
2,Second,21,209.925
3,Arde Bogotá,19,352.128
4,Vetusta Morla,18,1.115.111
5,Narco,17,55.234
6,Morgan,16,218.680
7,La M.O.D.A.,15,663.734
8,Sidonie,15,394.266
9,Dani Fernández,15,1.350.585
10,La Casa Azul,14,790.461


Grafico los resultados

In [126]:
import plotly.express as px

palette = ["#660000","#990033","#5F021F","#8C001A","#FF9000"]
palette_10 = ["#03071e","#370617","#6a040f","#9d0208","#d00000","#dc2f02","#e85d04","#f48c06","#faa307","#ffba08"]
values = artistas_top_10["escuchas_mensuales"]

fig = px.bar(artistas_top_10,
            x='artistas',
            y='cuenta_artistas')

fig.update_traces(
                    marker_color=np.tile(palette_10, len(artistas_analisis["artistas"])),
                    text=values,
                    textposition='auto',
                    textfont=dict(size=18))

fig.update_xaxes(
                tickangle=-45,
                tickfont=dict(size=16))

fig.update_yaxes(
                tickfont=dict(size=18),
                gridcolor='gray')


fig.update_layout(
                    width=1000,
                    height=600,
                    title_text="Top 10 grupos en festivales",
                    title_font=dict(size=30),
                    title_x=0.5, title_y=0.98,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    yaxis_title='Cantidad de festivales',
                    xaxis_title='Grupos',
                    xaxis_title_font=dict(size=25),
                    yaxis_title_font=dict(size=25),
                    font=dict(family='Century Gothic'),                           
                    showlegend=False)
                    
fig.add_annotation(x=9.4, y=0, xanchor='right', yanchor='top',
                    text='Fuente: fanmusicfest.com y Spotify',
                    font=dict(family='Century Gothic', size=16),
                    showarrow=False)  

fig.write_image(r'graficos\grafico_barras_grupos.png')
fig.show()

Creo un dataframe con las agrupaciones del tamaño de localidad

In [127]:
gb_tamaño_localidad = df_festivales_poblacion_002.groupby("tamaño_localidad")[["festival"]].count().reset_index().sort_values(ascending=False, by="festival")
gb_tamaño_localidad = gb_tamaño_localidad.rename(columns= {"festival": "total_festivales"})
gb_tamaño_localidad

,tamaño_localidad,total_festivales
2,10K - 50K,104
1,100K - 500K,90
6,50K - 100K,62
0,0 - 5K,53
7,5K - 10K,35
4,2M - 3.2M,30
5,500K - 1M,27
3,1M - 2M,13


Creo un diccionario con los datos del dataframe para que esteen ordenados por la columna tamaño_localidad (al ser un string no se ordena de la forma que quiero mostrar en el gráfico)

In [128]:
dict_tamaño_localidad = {"0- 5K": 53,
                        "5- 10K": 35,
                        "10- 50K": 104,
                        "50- 100K": 62,
                        "100- 500K": 90,
                        "500K- 1M": 27,
                        "1- 2M": 13,
                        "2- 3.2M": 30}

df_tamaño_localidad = pd.DataFrame(list(dict_tamaño_localidad.items()), columns=['tamaño_localidad', 'total_festivales'])
df_tamaño_localidad

,tamaño_localidad,total_festivales
0,0- 5K,53
1,5- 10K,35
2,10- 50K,104
3,50- 100K,62
4,100- 500K,90
5,500K- 1M,27
6,1- 2M,13
7,2- 3.2M,30


Muestro los resultados en un gráfico

In [129]:
import plotly.express as px

palette = ["#660000","#990033","#5F021F","#8C001A","#FF9000"]
palette_10 = ["#03071e","#370617","#6a040f","#9d0208","#d00000","#dc2f02","#e85d04","#f48c06","#faa307","#ffba08"]
values = df_tamaño_localidad["total_festivales"]

fig = px.bar(df_tamaño_localidad,
            x='tamaño_localidad',
            y='total_festivales')

fig.update_traces(
                    marker_color=np.tile(palette_10[::-1], len(df_tamaño_localidad["tamaño_localidad"])),
                    text=values,
                    textposition='auto',
                    textfont=dict(size=16))

fig.update_xaxes(
                tickangle=-45,
                tickfont=dict(size=18))

fig.update_yaxes(
                tickfont=dict(size=18),
                gridcolor='gray')


fig.update_layout(
                    width=1000,
                    height=600,
                    title_text="Total festivales por tamaño de población",
                    title_font=dict(size=30),
                    title_x=0.5, title_y=0.98,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    yaxis_title='Cantidad de festivales',
                    xaxis_title='Tamaño localidad',
                    yaxis_title_font=dict(size=25),
                    xaxis_title_font=dict(size=25),
                    font=dict(family='Century Gothic'),                           
                    showlegend=False)
                    
fig.add_annotation(x=7.4, y=0, xanchor='right', yanchor='top',
                    text='Fuente: fanmusicfest.com e INE.es',
                    font=dict(family='Century Gothic', size=16),
                    showarrow=False)

fig.write_image(r'graficos\grafico_barras_tamaño_localidad.png')
fig.show()

Creo un dataframe con los datos de cantidad de festivales por provincia (Top 10)

In [130]:
gb_provincia = df_festivales_poblacion_002.groupby("provincia")[["festival"]].count().reset_index().sort_values(ascending=False, by="festival")
gb_provincia = gb_provincia.rename(columns= {"festival": "total_festivales"})
gb_provincia.head(10)

,provincia,total_festivales
27,Madrid,37
6,Barcelona,29
29,Málaga,24
42,Valencia,21
2,Alicante,16
28,Murcia,15
37,Sevilla,14
21,Jaén,14
4,Asturias,13
20,Islas Baleares,12


Muestro los resultados en un gráfico

In [131]:
import plotly.express as px

palette = ["#660000","#990033","#5F021F","#8C001A","#FF9000"]
palette_10 = ["#03071e","#370617","#6a040f","#9d0208","#d00000","#dc2f02","#e85d04","#f48c06","#faa307","#ffba08"]
values = gb_provincia["total_festivales"]

fig = px.bar(gb_provincia.head(10),
            x='provincia',
            y='total_festivales')

fig.update_traces(
                    marker_color=np.tile(palette_10[::-1], len(gb_provincia["provincia"])),
                    text=values,
                    textposition='auto',
                    textfont=dict(size=16))

fig.update_xaxes(
                tickangle=-45,
                tickfont=dict(size=18))

fig.update_yaxes(
                tickfont=dict(size=18),
                gridcolor='gray')


fig.update_layout(
                    width=1000,
                    height=600,
                    title_text="Top 10 provincias en cantidad de festivales",
                    title_font=dict(size=30),
                    title_x=0.5, title_y=0.98,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    yaxis_title='Cantidad de festivales',
                    xaxis_title='Províncias',
                    yaxis_title_font=dict(size=25),
                    xaxis_title_font=dict(size=25),
                    font=dict(family='Century Gothic'),                           
                    showlegend=False)

fig.add_annotation(x=9.4, y=0, xanchor='right', yanchor='top',
                    text='Fuente: fanmusicfest.com',
                    font=dict(family='Century Gothic', size=16),
                    showarrow=False)                   

fig.write_image(r'graficos\grafico_barras_provincias.png')
fig.show()

Creo dataframe para ver la cantidad de festivales por comunidad autónoma

In [132]:
gb_comunidades = df_festivales_poblacion_002.groupby("comunidad_autonoma")[["festival"]].count().reset_index().sort_values(ascending=False, by="festival")
gb_comunidades = gb_comunidades.rename(columns= {"festival": "total_festivales"})
gb_comunidades

,comunidad_autonoma,total_festivales
0,Andalucía,79
8,Comunidad Valenciana,48
7,Cataluña,44
5,Castilla y León,40
13,Madrid,37
10,Galicia,26
16,País Vasco,20
6,Castilla-La Mancha,18
9,Extremadura,16
14,Murcia,15


Abrevio el nombre de algunas comunidades para que sno ocupen tanto espacio en la gráfica

In [133]:
for index, row in gb_comunidades.iterrows():
    if row["comunidad_autonoma"] == "Comunidad Valenciana":
        gb_comunidades.at[index, "comunidad_autonoma"] = "C. Valenciana"
    elif row["comunidad_autonoma"] == "Castilla y León":
        gb_comunidades.at[index, "comunidad_autonoma"] = "C. y León"
    elif row["comunidad_autonoma"] == "Castilla-La Mancha":
        gb_comunidades.at[index, "comunidad_autonoma"] = "C. La Mancha"
    elif row["comunidad_autonoma"] == "Islas Baleares":
        gb_comunidades.at[index, "comunidad_autonoma"] = "Baleares"

Muestro los resultados en el gráfico

In [134]:
import plotly.express as px

palette = ["#660000","#990033","#5F021F","#8C001A","#FF9000"]
palette_10 = ["#03071e","#370617","#6a040f","#9d0208","#d00000","#dc2f02","#e85d04","#f48c06","#faa307","#ffba08"]
values = gb_comunidades["total_festivales"]

fig = px.bar(gb_comunidades.head(10),
            x='comunidad_autonoma',
            y='total_festivales')

fig.update_traces(
                marker_color=np.tile(palette_10, len(gb_comunidades["comunidad_autonoma"])),
                text=values,
                textposition='auto',
                textfont=dict(size=16))

fig.update_xaxes(
                tickangle=-45,
                tickfont=dict(size=18))

fig.update_yaxes(
                tickfont=dict(size=18),
                gridcolor='gray')


fig.update_layout(
                    width=1000,
                    height=600,
                    title_text="Top 10 comunidades autónomas con mayor cantidad de festivales",
                    title_font=dict(size=30),
                    title_x=0.5, title_y=0.98,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    yaxis_title='Cantidad de festivales',
                    xaxis_title='Comunidades Autónomas',
                    yaxis_title_font=dict(size=25),
                    xaxis_title_font=dict(size=25),
                    font=dict(family='Century Gothic'),                           
                    showlegend=False)
                    
fig.add_annotation(x=9.4, y=0, xanchor='right', yanchor='top',
                    text='Fuente: fanmusicfest.com',
                    font=dict(family='Century Gothic', size=16),
                    showarrow=False)  

fig.write_image(r'graficos\grafico_barras_comunidades_head.png')
fig.show()

Creo un dataframe con las comunidades con menos festivales

In [135]:
gb_comunidades_tail = gb_comunidades.sort_values(by="total_festivales", ascending=True)
gb_comunidades_tail

,comunidad_autonoma,total_festivales
15,Navarra,5
12,La Rioja,6
4,Cantabria,10
11,Baleares,12
1,Aragón,12
3,Canarias,13
2,Asturias,13
14,Murcia,15
9,Extremadura,16
6,C. La Mancha,18


Grafico los resultados

In [136]:
import plotly.express as px

palette = ["#660000","#990033","#5F021F","#8C001A","#FF9000"]
palette_10 = ["#03071e","#370617","#6a040f","#9d0208","#d00000","#dc2f02","#e85d04","#f48c06","#faa307","#ffba08"]
values = gb_comunidades_tail["total_festivales"]

fig = px.bar(gb_comunidades_tail.head(5),
            x='comunidad_autonoma',
            y='total_festivales')

fig.update_traces(
                    marker_color=np.tile(palette_10, len(gb_comunidades["comunidad_autonoma"])),
                    text=values,
                    textposition='auto',
                    textfont=dict(size=16))


fig.update_xaxes(
                tickangle=-45,
                tickfont=dict(size=18))

fig.update_yaxes(
                tickfont=dict(size=18),
                gridcolor='gray')


fig.update_layout(
                    width=1000,
                    height=600,
                    title_text="Top 5 comunidades autónomas con menor cantidad de festivales",
                    title_font=dict(size=30),
                    title_x=0.5, title_y=0.98,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    yaxis_title='Cantidad de festivales',
                    xaxis_title='Comunidades Autónomas',
                    yaxis_title_font=dict(size=25),
                    xaxis_title_font=dict(size=25),
                    font=dict(family='Century Gothic'),                           
                    showlegend=False)
                    
fig.add_annotation(x=4.4, y=0, xanchor='right', yanchor='top',
                    text='Fuente: fanmusicfest.com',
                    font=dict(family='Century Gothic', size=16),
                    showarrow=False)  

fig.write_image(r'graficos\grafico_barras_comunidades_cola.png')
fig.show()

Creo dataframe para mostrar el ratio de festivales con respecto a si la provincia es costera o no

In [138]:
gb_costa = df_festivales_poblacion_002.groupby("costa")[["festival"]].count().reset_index().sort_values(ascending=False, by="festival")
gb_costa = gb_costa.rename(columns= {"festival": "total_festivales"})
gb_costa

,costa,total_festivales
1,True,241
0,False,173


Grafico los resultados

In [139]:
data = gb_costa
labels = ["Interior", "Costa"]
values = [241, 173]
palette = ["#6a040f","#f48c06"]

# palette = ["#03071e","#370617","#6a040f","#9d0208","#d00000","#dc2f02","#e85d04","#f48c06","#faa307","#ffba08"]

fig = px.pie(data,  
             values= values, 
             names= labels,
             hole= .5,
             color_discrete_sequence= palette)

fig.update_layout(
                    width=1000,
                    height=600,
                    title_text="Interior vs Costa",
                    title_font=dict(size=30),
                    title_x=0.5, title_y=0.95,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    yaxis_title_font=dict(size=25),
                    xaxis_title_font=dict(size=25),
                    font=dict(family='Century Gothic'))


fig.write_image(r'graficos\grafico_donut_interior_costa.png')
fig.show()

Creo dataframe para ver el reparto de los festivales a lo largo de los meses del año

In [143]:
gb_mes  = df_festivales_poblacion_002.groupby("mes")[["festival"]].count().reset_index().sort_values(ascending=False, by="mes")
gb_mes = gb_mes.rename(columns= {"festival": "total_festivales"})
gb_mes

,mes,total_festivales
11,Septiembre,36
10,Octubre,12
9,Noviembre,2
8,Mayo,38
7,Marzo,35
6,Junio,86
5,Julio,79
4,Febrero,19
3,Enero,12
2,Diciembre,1


Creo una función para renombrar los meses de forma que se ordenen a la hora de graficar. Después con Photoshop eliminaré los números de delante de los meses

In [144]:
def mes_orden(mes):
    if mes == "Enero":
        return '01 Enero'
    elif mes == "Febrero":
        return '02 Febrero'
    elif mes == "Marzo":
        return '03 Marzo'
    elif mes == "Abril":
        return '04 Abril'
    elif mes == "Mayo":
        return '05 Mayo'
    elif mes == "Junio":
        return '06 Junio'
    elif mes == "Julio":
        return '07 Julio'
    elif mes == "Agosto":
        return '08 Agosto'
    elif mes == "Septiembre":
        return '09 Septiembre'
    elif mes == "Octubre":
        return '10 Octubre'
    elif mes == "Noviembre":
        return '11 Noviembre'
    elif mes == "Diciembre":
        return '12 Diciembre'

In [145]:
gb_mes['mes'] = gb_mes['mes'].apply(lambda x: mes_orden(x))
gb_mes = gb_mes.sort_values(by= "mes", ascending= True)
gb_mes

,mes,total_festivales
3,01 Enero,12
4,02 Febrero,19
7,03 Marzo,35
0,04 Abril,43
8,05 Mayo,38
6,06 Junio,86
5,07 Julio,79
1,08 Agosto,51
11,09 Septiembre,36
10,10 Octubre,12


Muestro los resultados en la gráfica

In [146]:
import plotly.express as px

palette = ["#6a040f","#8C001A","#FF9000"]
palette_10 = ["#03071e","#370617","#6a040f","#9d0208","#d00000","#dc2f02","#e85d04","#f48c06","#faa307","#ffba08"]

fig = px.line(gb_mes,
            x="mes",
            y='total_festivales',
            color_discrete_sequence= palette,
            labels={"total_festivales": "cantidad"})

fig.update_traces(
            line=dict(width=4),
            mode='lines+markers',
            marker=dict(size=10))


fig.update_xaxes(
                tickangle=-45,
                tickfont=dict(size=18),
                showgrid=False)

fig.update_yaxes(
                tickfont=dict(size=18),
                gridcolor='gray')


fig.update_layout(
                    width=1000,
                    height=600,
                    title_text="Total festivales por meses",
                    title_font=dict(size=30),
                    title_x=0.5, title_y=0.98,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    yaxis_title='Cantidad',
                    xaxis_title='Meses',
                    yaxis_title_font=dict(size=25),
                    xaxis_title_font=dict(size=25),
                    font=dict(family='Century Gothic'),                           
                    showlegend=False)

fig.add_annotation(x=11.4, y=0, xanchor='right', yanchor='top',
                    text='Fuente: fanmusicfest.com',
                    font=dict(family='Century Gothic', size=16),
                    showarrow=False)                      

fig.write_image(r'graficos\grafico_lineas_festivales_meses.png')
fig.show()

Cambio el tipo de dato de la columna duracion para poder operar con los valores

In [147]:
df_festivales_poblacion_002["clasificacion_duracion"] = df_festivales_poblacion_002["duracion"].astype(int)

Creo una función para discretizar la variable duracion y englobar la duración de los festivales en 4 grupos

In [154]:
def clasificacion_duracion(duracion):
    if duracion == 3:
        return "3 días"
    if duracion == 1:
        return "1 día"
    if duracion == 2:
        return "2 días"
    elif duracion > 4:
        return "más de 3 días"
    
df_festivales_poblacion_002["clasificacion_duracion"] = df_festivales_poblacion_002["duracion"].apply(lambda x: clasificacion_duracion(x))

Visualizo los resultados para poder emplearlos en la gráfica

In [155]:
df_festivales_poblacion_002["clasificacion_duracion"].value_counts()

1 día            129
2 días           117
3 días            78
más de 3 días     61
Name: clasificacion_duracion, dtype: int64

Muestro los resultados en la gráfica

In [152]:
data = df_festivales_poblacion_002
labels = ['1 día', '2 días', '3 días', 'Más de 3 días']
values = [129, 117, 78, 61]
palette = ["#03071e","#370617","#6a040f","#9d0208"]

# palette = ["#03071e","#370617","#6a040f","#9d0208","#d00000","#dc2f02","#e85d04","#f48c06","#faa307","#ffba08"]

fig = px.pie(data,  
             values= values, 
             names= labels,
             hole= .5,
             color_discrete_sequence= palette,
             title='Gráfico de Pastel de Festivales por Comunidad Autónoma')

fig.update_layout(
                    width=1000,
                    height=600,
                    title_text="Duración de los festivales",
                    title_font=dict(size=30),
                    title_x=0.5, title_y=0.95,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    # yaxis_title='Cantidad',
                    # xaxis_title='Tamaño localidad',
                    yaxis_title_font=dict(size=25),
                    xaxis_title_font=dict(size=25),
                    font=dict(family='Century Gothic'))

fig.write_image(r'graficos\grafico_donut_duracion_festivales_001.png')
fig.show()

Visualizo los datos según la gratuidad de los festivales para crear una gráfica que lo muestre

In [ ]:
df_festivales_poblacion_002["gratuito"].value_counts()

False    339
True      34
Name: gratuito, dtype: int64

Grafico los resultados

In [153]:
data = df_festivales_poblacion_002
labels = ['Gratuito', 'De pago']
values = [34, 339]
# palette = ["#370617","#d00000"]

palette = ["#6a040f","#dc2f02"]

fig = px.pie(data,  
             values= values, 
             names= labels,
             hole= .5,
             color_discrete_sequence= palette,
             title='Gráfico de Pastel de Festivales por Comunidad Autónoma')

fig.update_layout(
                    width=1000,
                    height=600,
                    title_text="Gratuidad de los festivales",
                    title_font=dict(size=30),
                    title_x=0.5, title_y=0.95,
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)',
                    # yaxis_title='Cantidad',
                    # xaxis_title='Tamaño localidad',
                    yaxis_title_font=dict(size=25),
                    xaxis_title_font=dict(size=25),
                    font=dict(family='Century Gothic'))

fig.write_image(r'graficos\grafico_donut_gratuidad_festivales.png')
fig.show()